In [28]:
import os
import re
from tika import parser

In [29]:
pdf_dir = '/Users/eun-youngchristinapark/Documents/UBC-CAPSTONE/pdf_scraping/'
donquijote = 'Recorridos_Don-Quijote-Student-text-2017.pdf'
file_name = pdf_dir + donquijote

In [41]:
raw = parser.from_file(file_name)
raw_list = raw['content'].strip().split('\n\nRecorridos 3—Siglo XVI     Cervantes ')  ## contains pages of the book
text_list = []
regex = r'[0-9]+\. '
regex2 = r'[0-9]+\-'
regex3 = r'Capítulo [0-9]+  \n\n'
regex4 = r'[0-9]+\.'
regex5 = r'\n+\s+[0-9]+\s+\n+'
for i in range(11,84):
    if 'BANCO DE PALABRAS ÚTILES' not in raw_list[i]:  # skip End-of-Chapter Questions 
        if re.search(regex3, raw_list[i]) == None:
            begin_index = raw_list[i].find('\n\n')
            end_index = raw_list[i][:raw_list[i].find('=')+1].rfind('\n\n')
            end_index_prev = 0 
            if raw_list[i][begin_index:begin_index+30].startswith('\n\n¿COMPRENDISTE LO QUE LEÍSTE?'):  # we have to skip this section
                for match in re.finditer(regex4, raw_list[i]):
                    begin_index = match.span()[0]
                begin_index = raw_list[i].find('\n\n', begin_index + 1)
                end_index = raw_list[i][begin_index+1:raw_list[i].find('=')+1].rfind('\n\n') + begin_index+1
            elif begin_index == end_index:                                                 # the page starts with vocabulary list: we have to skip this section
                while end_index != -1:
                    end_index_prev += end_index + 1
                    end_index = raw_list[i][end_index_prev+1:].find('=')
                begin_index = raw_list[i].find('\n\n', end_index_prev + 1)
                end_index = 1
                search_begin_index = begin_index 
                while end_index > 0:
                    regex_match = re.search(regex5, raw_list[i][search_begin_index + 1:])
                    if regex_match:
                        end_index = regex_match.span()[0]
                        search_begin_index += end_index 
                    else:
                        end_index = -1 
                end_index = search_begin_index
        else:
            begin_index = re.search(regex3, raw_list[i]).span()[0]
            end_index = raw_list[i][begin_index+1:raw_list[i].find('=')+1].rfind('\n\n') + begin_index+1
        
        
        ts = raw_list[i][begin_index:end_index]   # main text 
        ts_list = ts.split('\n')
        ts_list_filtered = []
        qu_list = [0] * len(ts_list)
        for idx, sent in enumerate(ts_list):
            qu_list[idx] = 0
            if sent.strip() == '' and idx > 0:                                                                        # remove empty strings 
                qu_list[idx] = qu_list[idx-1]
            elif sent.strip().startswith('¿') or re.match(regex, sent.strip()) or re.match(regex2, sent.strip()):     # textbook questions or comments
                if sent.strip().endswith('.') or sent.strip().endswith('?') or re.match(regex2, sent.strip()):
                    qu_list[idx] = 1                                                                                  # question started and ended
                else:
                    qu_list[idx] = 2                                                                                  # question started (not ended)
            elif idx > 0 and (qu_list[idx-1] == 2 or qu_list[idx-1] == 3):
                if sent.strip().endswith('.') or sent.strip().endswith('?'):
                    qu_list[idx] = 4                                                                                  # question ended
                else:
                    qu_list[idx] = 3                                                                                  # question continues 
    
            elif sent.strip().startswith('“'):
                if sent.strip().endswith('”'):
                    qu_list[idx] = 5                                                                                  # quotation started and ended
                else:
                    qu_list[idx] = 6                                                                                  # quotation started (not ended)
            elif idx > 0 and qu_list[idx-1] == 6 and not(sent.strip().endswith('”')):
                qu_list[idx] = 7                                                                                      # quotatoin continues
            elif idx > 0 and (qu_list[idx-1] == 6 or qu_list[idx-1] == 7) and sent.strip().endswith('”'):
                qu_list[idx] = 8                                                                                      # quotation ended 
            if idx > 0 and qu_list[idx-1] == 8:
                qu_list[idx] = 9
        
            if qu_list[idx] == 0:
                ts_list_filtered.append(sent)
        ts_filtered = '\n'.join(ts_list_filtered)
        text_list.append(ts_filtered)

In [68]:
i = 25
print(text_list[i].strip())

Un mozo de mulas de los que allí venían, que no debía de ser 
muy bien intencionado, oyendo decir al pobre caído tantas    
arrogancias, no lo pudo sufrir sin darle la respuesta en las 
costillas. Y, llegándose a él, tomó la lanza y, después de haberla 
hecho pedazos, con uno dellos comenzó a dar a nuestro don 
Quijote tantos palos que, a despecho y pesar de sus armas, le molió 
como cibera. Débanle voces sus amos que no le diese tanto y que 
le dejase, pero estaba ya el mozo picado y no quiso dejar el juego 
hasta envidar todo el resto de su cólera; y, acudiendo por los 
demás trozos de la lanza, los acabó de deshacer sobre el miserable 
caído, que, con toda aquella tempestad de palos que sobre él 
vía, no cerraba la boca, amenazando al cielo y a la tierra, y a los 
malandrines, que tal le parecían. 
 
  Cansóse el mozo, y los mercaderes siguieron su camino, 
llevando qué contar en todo él del pobre apaleado. El cual, después 
que se vio solo, tornó a probar si podía levantarse; pero 

In [31]:
i = 13
if 'BANCO DE PALABRAS ÚTILES' not in raw_list[i]:  # skip End-of-Chapter Questions 
    if re.search(regex3, raw_list[i]) == None:
        begin_index = raw_list[i].find('\n\n')
        end_index = raw_list[i][:raw_list[i].find('=')+1].rfind('\n\n')
        end_index_prev = 0 
        if raw_list[i][begin_index:begin_index+30].startswith('\n\n¿COMPRENDISTE LO QUE LEÍSTE?'):  # we have to skip this section
            for match in re.finditer(regex4, raw_list[i]):
                begin_index = match.span()[0]
            begin_index = raw_list[i].find('\n\n', begin_index + 1)
            end_index = raw_list[i][begin_index+1:raw_list[i].find('=')+1].rfind('\n\n') + begin_index+1
        elif begin_index == end_index:                                                 # the page starts with vocabulary list: we have to skip this section
            while end_index != -1:
                end_index_prev += end_index + 1
                end_index = raw_list[i][end_index_prev+1:].find('=')
            begin_index = raw_list[i].find('\n\n', end_index_prev + 1)
            end_index = 1
            search_begin_index = begin_index 
            while end_index > 0:
                regex_match = re.search(regex5, raw_list[i][search_begin_index + 1:])
                if regex_match:
                    end_index = regex_match.span()[0]
                    search_begin_index += end_index 
                else:
                    end_index = -1 
            end_index = search_begin_index
    else:
        begin_index = re.search(regex3, raw_list[i]).span()[0]
        end_index = raw_list[i][begin_index+1:raw_list[i].find('=')+1].rfind('\n\n') + begin_index+1     

In [32]:
print(raw_list[13][begin_index:end_index])



caballero melindroso, ni tan llorón como su hermano, y que en lo 
de la valentía no le iba en zaga. 
 
 En resolución, él se enfrascó tanto en su lectura, que se 
le pasaban las noches leyendo de claro en claro, y los días de     
turbio en turbio; y así, del poco dormir y del mucho leer, se  
le secó el cerebro, de manera que vino a perder el juicio.      
Llenósele la fantasía de todo aquello que leía en los libros, así de 
encantamientos como de pendencias, batallas, desafíos, heridas, 
requiebros, amores, tormentas y disparates imposibles; y 
asentósele de tal modo en la imaginación que era verdad toda 
aquella máquina de aquellas sonadas soñadas invenciones que 
leía, que para él no había otra historia más cierta en el mundo. 
Decía él que el Cid Ruy Díaz había sido muy buen caballero,   
pero que no tenía que ver con el Caballero de la Ardiente 
Espada, que de sólo un revés había partido por medio dos  
fieros y descomunales gigantes. Mejor estaba con Bernardo 
del Carpio, porq

In [39]:
i = 13
ts = raw_list[i][begin_index:end_index]   # main text 
ts_list = ts.split('\n')
ts_list_filtered = []
qu_list = [0] * len(ts_list)
for idx, sent in enumerate(ts_list):
    qu_list[idx] = 0
    if sent.strip() == '' and idx > 0:                                                                        # remove empty strings 
        qu_list[idx] = qu_list[idx-1]
    elif sent.strip().startswith('¿') or re.match(regex, sent.strip()) or re.match(regex2, sent.strip()):     # textbook questions or comments
        if sent.strip().endswith('.') or sent.strip().endswith('?') or re.match(regex2, sent.strip()):
            qu_list[idx] = 1                                                                                  # question started and ended
        else:
            qu_list[idx] = 2                                                                                  # question started (not ended)
    elif idx > 0 and (qu_list[idx-1] == 2 or qu_list[idx-1] == 3):
        if sent.strip().endswith('.') or sent.strip().endswith('?'):
            qu_list[idx] = 4                                                                                  # question ended
        else:
            qu_list[idx] = 3                                                                                  # question continues 
    
    elif sent.strip().startswith('“'):
        if sent.strip().endswith('”'):
            qu_list[idx] = 5                                                                                  # quotation started and ended
        else:
            qu_list[idx] = 6                                                                                  # quotation started (not ended)
    elif idx > 0 and qu_list[idx-1] == 6 and not(sent.strip().endswith('”')):
        qu_list[idx] = 7                                                                                      # quotatoin continues
    elif idx > 0 and (qu_list[idx-1] == 6 or qu_list[idx-1] == 7) and sent.strip().endswith('”'):
        qu_list[idx] = 8                                                                                      # quotation ended 
    if idx > 0 and qu_list[idx-1] == 8:
        qu_list[idx] = 9
        
    if qu_list[idx] == 0:
        ts_list_filtered.append(sent)
ts_filtered = '\n'.join(ts_list_filtered)
text_list.append(ts_filtered)

In [40]:
for i, sent in enumerate(ts_list):
    print(i, sent, qu_list[i])

0  0
1  0
2 caballero melindroso, ni tan llorón como su hermano, y que en lo  0
3 de la valentía no le iba en zaga.  0
4   0
5  En resolución, él se enfrascó tanto en su lectura, que se  0
6 le pasaban las noches leyendo de claro en claro, y los días de      0
7 turbio en turbio; y así, del poco dormir y del mucho leer, se   0
8 le secó el cerebro, de manera que vino a perder el juicio.       0
9 Llenósele la fantasía de todo aquello que leía en los libros, así de  0
10 encantamientos como de pendencias, batallas, desafíos, heridas,  0
11 requiebros, amores, tormentas y disparates imposibles; y  0
12 asentósele de tal modo en la imaginación que era verdad toda  0
13 aquella máquina de aquellas sonadas soñadas invenciones que  0
14 leía, que para él no había otra historia más cierta en el mundo.  0
15 Decía él que el Cid Ruy Díaz había sido muy buen caballero,    0
16 pero que no tenía que ver con el Caballero de la Ardiente  0
17 Espada, que de sólo un revés había partido por medio dos

In [25]:
i = 13
ts = raw_list[i][begin_index:end_index]   # main text 
ts_list = ts.split('\n')
ts_list_filtered = []
qu_list = [0] * len(ts_list)
for idx, sent in enumerate(ts_list):
    qu_list[idx] = 0
    if sent.strip() == '' and idx > 0:                                                                        # remove empty strings 
        qu_list[idx] = qu_list[idx-1]
    elif sent.strip().startswith('¿') or re.match(regex, sent.strip()) or re.match(regex2, sent.strip()):     # textbook questions or comments
        if sent.strip().endswith('.') or sent.strip().endswith('?'):
            qu_list[idx] = 1                                                                                  # question started and ended
        else:
            qu_list[idx] = 2                                                                                  # question started (not ended)
    elif idx > 0 and (qu_list[idx-1] == 2 or qu_list[idx-1] == 3):
        if sent.strip().endswith('.') or sent.strip().endswith('?'):
            qu_list[idx] = 4                                                                                  # question ended
        else:
            qu_list[idx] = 3                                                                                  # question continues 
    
    elif sent.strip().startswith('“'):
        if sent.strip().endswith('”'):
            qu_list[idx] = 5                                                                                  # quotation started and ended
        else:
            qu_list[idx] = 6                                                                                  # quotation started (not ended)
    elif idx > 0 and qu_list[idx-1] == 6 and not(sent.strip().endswith('”')):
        qu_list[idx] = 7                                                                                      # quotatoin continues
    elif idx > 0 and (qu_list[idx-1] == 6 or qu_list[idx-1] == 7) and sent.strip().endswith('”'):
        qu_list[idx] = 8                                                                                      # quotation ended 
    if idx > 0 and qu_list[idx-1] == 8:
        qu_list[idx] = 9
        
    if qu_list[idx] == 0:
        ts_list_filtered.append(sent)
    ts_filtered = '\n'.join(ts_list_filtered)
    text_list.append(ts_filtered)

In [27]:
print(ts)



caballero melindroso, ni tan llorón como su hermano, y que en lo 
de la valentía no le iba en zaga. 
 
 En resolución, él se enfrascó tanto en su lectura, que se 
le pasaban las noches leyendo de claro en claro, y los días de     
turbio en turbio; y así, del poco dormir y del mucho leer, se  
le secó el cerebro, de manera que vino a perder el juicio.      
Llenósele la fantasía de todo aquello que leía en los libros, así de 
encantamientos como de pendencias, batallas, desafíos, heridas, 
requiebros, amores, tormentas y disparates imposibles; y 
asentósele de tal modo en la imaginación que era verdad toda 
aquella máquina de aquellas sonadas soñadas invenciones que 
leía, que para él no había otra historia más cierta en el mundo. 
Decía él que el Cid Ruy Díaz había sido muy buen caballero,   
pero que no tenía que ver con el Caballero de la Ardiente 
Espada, que de sólo un revés había partido por medio dos  
fieros y descomunales gigantes. Mejor estaba con Bernardo 
del Carpio, porq

In [26]:
print(ts_filtered)



caballero melindroso, ni tan llorón como su hermano, y que en lo 
de la valentía no le iba en zaga. 
 
 En resolución, él se enfrascó tanto en su lectura, que se 
le pasaban las noches leyendo de claro en claro, y los días de     
turbio en turbio; y así, del poco dormir y del mucho leer, se  
le secó el cerebro, de manera que vino a perder el juicio.      
Llenósele la fantasía de todo aquello que leía en los libros, así de 
encantamientos como de pendencias, batallas, desafíos, heridas, 
requiebros, amores, tormentas y disparates imposibles; y 
asentósele de tal modo en la imaginación que era verdad toda 
aquella máquina de aquellas sonadas soñadas invenciones que 
leía, que para él no había otra historia más cierta en el mundo. 
Decía él que el Cid Ruy Díaz había sido muy buen caballero,   
pero que no tenía que ver con el Caballero de la Ardiente 
Espada, que de sólo un revés había partido por medio dos  
fieros y descomunales gigantes. Mejor estaba con Bernardo 
del Carpio, porq

In [323]:
sent = ts_list[25]
idx = 25
if idx> 0 and qu_list[idx-1] == 1 and not(ts_list[idx-1].endswith('.')) and not(ts_list[idx-1].endswith('?')):
    qu_list[idx] = 3
qu_list[24]

3

In [321]:
for i, sent in enumerate(ts_list):
    print(i, sent)

0 
1 
2 don Quijote palabra y sin pedir favor a nadie, soltó otra vez la 
3 adarga y alzó otra vez la lanza y, sin hacerla pedazos, hizo más de 
4 tres la cabeza del segundo arriero, porque se la abrió por cuatro. 
5 Al ruido acudió toda la gente de la venta, y entre ellos el ventero. 
6 Viendo esto don Quijote, embrazó su adarga y, puesta mano a su 
7 espada, dijo: 
8  
9  —¡Oh señora de la fermosura, esfuerzo y vigor del debilitado 
10 corazón mío! Ahora es tiempo que vuelvas los ojos de tu grandeza 
11 a este tu cautivo caballero, que tamaña aventura está atendiendo. 
12 
13   
14 
15  
16 
17  
18 
19 ¿COMPRENDISTE LO QUE LEÍSTE? 3-A 
20 29. Describe cómo don Quijote le pide al ventero que lo arme caballero. 
21 30. Explica cómo reacciona el ventero a su petición. 
22 31. ¿Dónde quiere velar las armas don Quijote? ¿Por qué?
23 32. Cita dónde Cervantes nos indica que el ventero le sigue el humor (o la corriente) a don  
24 
25 Quijote y explica. 
26 
27 “…asió de su lanza y con gent

In [318]:
print(ts_filtered)



don Quijote palabra y sin pedir favor a nadie, soltó otra vez la 
adarga y alzó otra vez la lanza y, sin hacerla pedazos, hizo más de 
tres la cabeza del segundo arriero, porque se la abrió por cuatro. 
Al ruido acudió toda la gente de la venta, y entre ellos el ventero. 
Viendo esto don Quijote, embrazó su adarga y, puesta mano a su 
espada, dijo: 
 
 —¡Oh señora de la fermosura, esfuerzo y vigor del debilitado 
corazón mío! Ahora es tiempo que vuelvas los ojos de tu grandeza 
a este tu cautivo caballero, que tamaña aventura está atendiendo. 

  

 

 

Quijote y explica. 



In [305]:
for i, sent in enumerate(ts_list):
    print(i, sent)

0 
1 
2 don Quijote palabra y sin pedir favor a nadie, soltó otra vez la 
3 adarga y alzó otra vez la lanza y, sin hacerla pedazos, hizo más de 
4 tres la cabeza del segundo arriero, porque se la abrió por cuatro. 
5 Al ruido acudió toda la gente de la venta, y entre ellos el ventero. 
6 Viendo esto don Quijote, embrazó su adarga y, puesta mano a su 
7 espada, dijo: 
8  
9  —¡Oh señora de la fermosura, esfuerzo y vigor del debilitado 
10 corazón mío! Ahora es tiempo que vuelvas los ojos de tu grandeza 
11 a este tu cautivo caballero, que tamaña aventura está atendiendo. 
12 
13   
14 
15  
16 
17  
18 
19 ¿COMPRENDISTE LO QUE LEÍSTE? 3-A 
20 29. Describe cómo don Quijote le pide al ventero que lo arme caballero. 
21 30. Explica cómo reacciona el ventero a su petición. 
22 31. ¿Dónde quiere velar las armas don Quijote? ¿Por qué?
23 32. Cita dónde Cervantes nos indica que el ventero le sigue el humor (o la corriente) a don  
24 
25 Quijote y explica. 
26 
27 “…asió de su lanza y con gent

In [245]:
i = 29
regex4 = r'[0-9]+\.'
regex5 = r'\n+\s+[0-9]+\s+\n+'
if 'BANCO DE PALABRAS ÚTILES' not in raw_list[i]:  # skip End-of-Chapter Questions 
        if re.search(regex3, raw_list[i]) == None:
            begin_index = raw_list[i].find('\n\n')
            end_index = raw_list[i][:raw_list[i].find('=')+1].rfind('\n\n')
            end_index_prev = 0
            if raw_list[i][begin_index:begin_index+30].startswith('\n\n¿COMPRENDISTE LO QUE LEÍSTE?'):  # we have to skip this section
                for match in re.finditer(regex4, raw_list[i]):
                    begin_index = match.span()[0]
                begin_index = raw_list[i].find('\n\n', begin_index + 1)
            elif begin_index == end_index : # starts with vocabulary list 
                while end_index != -1:
                    end_index_prev += end_index + 1
                    end_index = raw_list[i][end_index_prev+1:].find('=') 
                    #print(f'end_index:{ end_index}, end_index_prev: {end_index_prev}')
                begin_index = raw_list[i].find('\n\n', end_index_prev + 1)
                end_index = 1
                search_begin_index = begin_index 
                while end_index > 0:
                    regex_match = re.search(regex5, raw_list[i][search_begin_index + 1:])
                    if regex_match:
                        end_index = regex_match.span()[0]
                        search_begin_index += end_index 
                    else:
                        end_index = -1        
        else:
            begin_index = re.search(regex3, raw_list[i]).span()[0]

In [292]:
i = 14
index = 0
find_count = 0
find_index = 0
search_index = 0
while find_index >= 0:
    find_index = raw_list[i].find('Capítulo', search_index + 1)
    search_index += find_index
    if find_index > 0:
        find_count += 1
    #print(find_index)
    #print(search_index)
print(find_count)

1


In [297]:
raw_list[29].find('Capítulo', search_index + 1)

-1

In [298]:
raw_list[29]

'26   \n\ndon Quijote palabra y sin pedir favor a nadie, soltó otra vez la \nadarga y alzó otra vez la lanza y, sin hacerla pedazos, hizo más de \ntres la cabeza del segundo arriero, porque se la abrió por cuatro. \nAl ruido acudió toda la gente de la venta, y entre ellos el ventero. \nViendo esto don Quijote, embrazó su adarga y, puesta mano a su \nespada, dijo: \n \n —¡Oh señora de la fermosura, esfuerzo y vigor del debilitado \ncorazón mío! Ahora es tiempo que vuelvas los ojos de tu grandeza \na este tu cautivo caballero, que tamaña aventura está atendiendo. \n\n  \n\n \n\n \n\n¿COMPRENDISTE LO QUE LEÍSTE? 3-A \n29. Describe cómo don Quijote le pide al ventero que lo arme caballero. \n30. Explica cómo reacciona el ventero a su petición. \n31. ¿Dónde quiere velar las armas don Quijote? ¿Por qué?\n32. Cita dónde Cervantes nos indica que el ventero le sigue el humor (o la corriente) a don  \n\nQuijote y explica. \n\n“…asió de su lanza y con gentil continente se comenzó a \npasear delan

In [299]:
print(text_list[16])

don Quijote palabra y sin pedir favor a nadie, soltó otra vez la 
adarga y alzó otra vez la lanza y, sin hacerla pedazos, hizo más de 
tres la cabeza del segundo arriero, porque se la abrió por cuatro. 
Al ruido acudió toda la gente de la venta, y entre ellos el ventero. 
Viendo esto don Quijote, embrazó su adarga y, puesta mano a su 
espada, dijo: 
 —¡Oh señora de la fermosura, esfuerzo y vigor del debilitado 
corazón mío! Ahora es tiempo que vuelvas los ojos de tu grandeza 
a este tu cautivo caballero, que tamaña aventura está atendiendo. 
Quijote y explica. 
“…asió de su lanza y con gentil continente se comenzó a 
pasear delante de la pila…”      
                                              Don Quijote, Capítulo III 
MONEDAS DEL REINO DE NAVARRA 
 Cervantes compara los cuartos de un real a 
la apariencia del rocín de Alonso 
Quijano. 
a don Quijote? 
don Quijote las armas y el
conflicto que resulta.  


In [139]:
raw_list[28]

'25   \n\ncorral=sitio para guardar animales \npila=pieza de piedra donde se pone agua  \nasió=agarró  \ncontinente=compostura \n\narmazón=de armar \n\nsosegado ademán=gesto tranquilo\narrimado=juntado a \n\nAntojósele=Tuvo ganas de \nrecua=grupo de animales de carga \nrespecto a sus padrinos \n\n \n\nse ciñó=se ajustó  \n \nse curó=se preocupó \n \ncorreas=cinturones  \ntrecho=lejos  \n \n \nAcorredme=Ayúdame  \navasallado=sumiso  \ndesfallezca=falle  \ntrance=combate  \namparo=protección  \n \nmaltrecho=herido  \nmaestro=médico \n \n \naturdido=confundido \n \ndesembarazar=quitar obstáculos\n\n Prometióle don Quijote de hacer lo que se le aconsejaba \ncon toda puntualidad; y así, se dio luego orden como velase las \narmas en un corral grande que a un lado de la venta estaba; y, \nrecogiéndolas don Quijote todas, las puso sobre una pila que \njunto a un pozo estaba y, embrazando su adarga, asió de su lanza \ny con gentil continente se comenzó a pasear delante de la pila; y \ncuando co

In [17]:
i = 2 # 15 for page 25 
print(text_list[i])



caballero melindroso, ni tan llorón como su hermano, y que en lo 
de la valentía no le iba en zaga. 
 
 En resolución, él se enfrascó tanto en su lectura, que se 
le pasaban las noches leyendo de claro en claro, y los días de     
turbio en turbio; y así, del poco dormir y del mucho leer, se  
le secó el cerebro, de manera que vino a perder el juicio.      
Llenósele la fantasía de todo aquello que leía en los libros, así de 
encantamientos como de pendencias, batallas, desafíos, heridas, 
requiebros, amores, tormentas y disparates imposibles; y 
asentósele de tal modo en la imaginación que era verdad toda 
aquella máquina de aquellas sonadas soñadas invenciones que 
leía, que para él no había otra historia más cierta en el mundo. 
Decía él que el Cid Ruy Díaz había sido muy buen caballero,   
pero que no tenía que ver con el Caballero de la Ardiente 
Espada, que de sólo un revés había partido por medio dos  
fieros y descomunales gigantes. Mejor estaba con Bernardo 
del Carpio, porq

In [195]:
i = 16
begin_index = raw_list[i].find('\n\n')
end_index = raw_list[i][:raw_list[i].find('=')+1].rfind('\n\n')
ts = raw_list[i][begin_index:end_index]
ts_list = ts.split('\n')
ts_list_filtered = []
qu_list = [0] * len(ts_list)
for idx, sent in enumerate(ts_list):
    qu_list[idx] = 0
    if sent.strip().startswith('¿'):
        qu_list[idx] = 1
    if re.match(regex, sent.strip()):
        qu_list[idx] = 1
    if re.match(regex2, sent.strip()):
        qu_list[idx] = 1
    if sent.strip() == '':
        qu_list[idx] = 1
    if idx> 0 and qu_list[idx-1] == 1 and not(ts_list[idx-1].endswith('.')) and not(ts_list[idx-1].endswith('?')):
        qu_list[idx] = 2
        
    if qu_list[idx] == 0:
        print(idx, sent)
        ts_list_filtered.append(sent)

8 le da? 
11 compuesta celada, embrazó su adarga, tomó su lanza y, por la puerta falsa de un 
12 corral, salió al campo con grandísimo contento…”  
17 hecho este discurso, y más cuando halló a quien dar nombre de 
18 su dama! Y fue, a lo que se cree, que en un lugar cerca del suyo 
19 había una moza labradora de muy buen parecer, de quien él un 
20 tiempo anduvo enamorado, aunque, según se entiende, ella jamás 
21 lo supo, ni le dio cata dello. Llamábase Aldonza Lorenzo, y a 
22 ésta le pareció ser bien darle título de señora de sus pensamientos; 
23 y, buscándole nombre que no desdijese mucho del suyo, y que 
24 tirase y se encaminase al de princesa y gran señora, vino a lla marla 
25 Dulcinea del Toboso, porque era natural del Toboso; nombre, a su 
26 parecer, músico y peregrino y significativo, como todos los demás 
27 que a él y a sus cosas había puesto. 


In [193]:
ts_list

['',
 '',
 '¿COMPRENDISTE LO QUE LEÍSTE? 1-D ',
 '12. ¿Cuánto tiempo dura en ponerse un nombre adecuado para su nuevo oficio? ¿Quién le ',
 '',
 'inspira al hidalgo a llamarse don Quijote de la Mancha?  ',
 '13. Describe a la señora de sus pensamientos. ¿Cómo justifica el hidalgo el nombre que ',
 '',
 'le da? ',
 '',
 '“…se armó de todas sus armas, subió sobre Rocinante, puesta su mal ',
 'compuesta celada, embrazó su adarga, tomó su lanza y, por la puerta falsa de un ',
 'corral, salió al campo con grandísimo contento…”  ',
 '',
 '          Don Quijote, El Ingenioso hidalgo don Quijote de la Mancha, Capítulo 2  ',
 '',
 ' ¡Oh, cómo se holgó nuestro buen caballero cuando hubo ',
 'hecho este discurso, y más cuando halló a quien dar nombre de ',
 'su dama! Y fue, a lo que se cree, que en un lugar cerca del suyo ',
 'había una moza labradora de muy buen parecer, de quien él un ',
 'tiempo anduvo enamorado, aunque, según se entiende, ella jamás ',
 'lo supo, ni le dio cata dello. Llamába

In [179]:
ts_list_filtered

['',
 '',
 '',
 'deseaba.',
 ' Puesto nombre, y tan a su gusto, a su caballo, quiso  ',
 'ponérsele a sí mismo, y en este pensamiento duró otros ocho  ',
 'días, y al cabo se vino a llamar don Quijote; de donde, como  ',
 'queda dicho, tomaron ocasión los autores desta tan verdadera ',
 'historia que, sin duda, se debía de llamar Quijada, y no Quesada, ',
 'como otros quisieron decir. Pero, acordándose que el valeroso  ',
 'Amadís no sólo se había contentado con llamarse Amadís  ',
 'a secas, sino que añadió el nombre de su reino y patria, por ',
 'Hepila famosa, y se llamó Amadís de Gaula, así quiso, como ',
 'buen caballero, añadir al suyo el nombre de la suya y llamarse ',
 'don Quijote de la Mancha, con que, a su parecer, declaraba muy al  ',
 'vivo su linaje y patria, y la honraba con tomar el sobrenombre  ',
 'della. ',
 '                  ',
 ' Limpias, pues, sus armas, hecho del morrión celada, ',
 'puesto nombre a su rocín y confirmándose a sí mismo, se dio a ',
 'entender que

In [174]:
qu_list

[0,
 0,
 1,
 2,
 0,
 0,
 1,
 2,
 1,
 2,
 1,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [148]:
begin_index = raw_list[13].find('\n\n')
end_index = raw_list[13][:raw_list[13].find('=')+1].rfind('\n\n')
ts = raw_list[13][begin_index:end_index]
ts_list = ts.split('\n')
len(ts_list)

47

In [149]:
ts_list

['',
 '',
 'caballero melindroso, ni tan llorón como su hermano, y que en lo ',
 'de la valentía no le iba en zaga. ',
 ' ',
 ' En resolución, él se enfrascó tanto en su lectura, que se ',
 'le pasaban las noches leyendo de claro en claro, y los días de     ',
 'turbio en turbio; y así, del poco dormir y del mucho leer, se  ',
 'le secó el cerebro, de manera que vino a perder el juicio.      ',
 'Llenósele la fantasía de todo aquello que leía en los libros, así de ',
 'encantamientos como de pendencias, batallas, desafíos, heridas, ',
 'requiebros, amores, tormentas y disparates imposibles; y ',
 'asentósele de tal modo en la imaginación que era verdad toda ',
 'aquella máquina de aquellas sonadas soñadas invenciones que ',
 'leía, que para él no había otra historia más cierta en el mundo. ',
 'Decía él que el Cid Ruy Díaz había sido muy buen caballero,   ',
 'pero que no tenía que ver con el Caballero de la Ardiente ',
 'Espada, que de sólo un revés había partido por medio dos  ',
 'f

In [131]:
ts_list[31].strip().startswith('¿')

True

In [134]:
regex = r'[0-9]+\. '
re.match(regex, ts_list[42].strip())

<re.Match object; span=(0, 3), match='5. '>

In [132]:
ts_list[42].strip()

'5.  ¿Con quiénes discute el hidalgo las obras'

In [68]:
raw_list[11][:raw_list[11].find('=')+1].rfind('\n\n')

1450

In [69]:
raw_list[11][:1450]

'8   \n\nCapítulo I \nQue trata de la condición y ejercicio del famoso\nhi dalgo don Quijote de la Mancha  \n                                                                   \n\n                                                           \n de cuyo nombre no \nquiero acordarme, no ha mucho tiempo que vivía un hidalgo \nde los de lanza en astillero, adarga antigua, rocín flaco y galgo \ncorredor. Una olla de algo más vaca que carnero, salpicón las \nmás noches, duelos y quebrantos los sábados, lentejas los \nviernes, algún palomino de añadidura los domingos, consumían \nlas tres partes de su hacienda. El resto della concluían sayo \nde velarte, calzas de velludo para las fiestas, con sus pantu-\nflos de lo mesmo, y los días de entresemana se honraba consu \nvellorí de lo más fino. Tenía en su casa una ama que pasaba  \nde los cuarenta, y una sobrina que no llegaba a los veinte, y \nun mozo de campo y plaza, que así ensillaba el rocín como \ntomaba la podadera. Frisaba la edad de nuestr